# Sub Question

The current mesh is not capable of capturing the required details.



# 3D Mesh Generation Overall Approach
#### [Relevant Notebook](./ensemble/sampled/mesh_generation.ipynb)

1. Compute normals for each timeslice group: 
Normals are oriented  with respect to the input point cloud if normals exist. Next, converts float64 numpy array of shape (n, 3) to Open3D format. Normals are required to generate meshes.

2. Generate Poission Mesh: (See Screened Poission Reconstruction for more information below)
Implements the Screened Poisson Reconstruction proposed in Kazhdan and Hoppe, "Screened Poisson Surface Reconstruction", 2013. See https://github.com/mkazhdan/PoissonRecon

3. Save as `.off` files
The network specifically handles `.off` files
    
#### Screened Poisson Reconstruction
It uses an approach known as an implicit meshing method, which is trying to “envelop” the data in a smooth cloth. We try to fit a watertight surface from the original point set by creating an entirely new point set representing an isosurface linked to the normals. There are several parameters available that affect the result of the meshing:

1. Depth:
Tree-depth is used for the reconstruction. The higher, the more detailed the mesh. With noisy data you keep vertices in the generated mesh that are outliers but the algorithm doesn’t detect them as such. So a low value (maybe between 5 and 7) provides a smoothing effect, but you will lose detail. The higher the depth-value the higher is the resulting amount of vertices of the generated mesh.

2. Width: 
This specifies the target width of the finest level of the tree structure, which is called an octree. D

3. Scale:
It describes the ratio between the diameter of the cube used for reconstruction and the diameter of the samples’ bounding cube. Very abstract, the default parameter usually works well.

4. Fit:
The linear_fit parameter if set to true, let the reconstructor use linear interpolation to estimate the positions of iso-vertices.

# Problem At Hand
(Identified on 22-August-2020)

When sampling 6550 points only per timeslice for both train and test ie. using a small subset of data per timeslice, there is distinct difference between the two types of classes, allowing for the classifier to classify with great accuraccy.

![](assets/experiments/problem/noise_6550.png)
![](assets/experiments/problem/mixed_6550.png)

However, without this sampling, the meshes look very similar to each other. 
![](assets/experiments/problem/noise_unsampled.png)
![](assets/experiments/problem/mixed_unsampled.png)

# BPA

# BPA + Lapalacian Smoothning

# Radius Based Outlier Detection with Clustering Verification

First check how many clusters are found in the data:
1. If cluster > 0:
    1. Compute average radius
    2. Remove outliers based on radius
    
2. If cluster == 0:
    1. No removal of points
    
**Note:**
1. For timeslice with only 3 points, it takes `eps` = 200 to identify. (default = 100)

#### Clustering
DBScan based cluster detection

#### Radius based outlier detection
Simple average distance between each point is calculated using k-means, and then multiplied by 3.

## Group with Noise + Hits
The DBScan clustering algorithm shows two clusters (indicated by arrows)
![](../../assets/experiments/mesh/mixed_cluster.png)

Using radius based outlier detection, the red points are all identified as outliers while gray points are all inliers
![](../../assets/experiments/mesh/mixed_outlier.png)

The point cloud now gets processed to remove all outlier points
![](../../assets/experiments/mesh/mixed_processed.png)

Finally, with the poission mesh recomputed
![](../../assets/experiments/mesh/mixed_mesh.png)


## Group with Noise
The DBScan clustering algorithm shows no clusters 
![](../../assets/experiments/mesh/noise_cluster.png)

The poission mesh recomputed
![](../../assets/experiments/mesh/noise_mesh.png)


Therefore no action is taken and pcl is converted to mesh.

## Anomaly: `Group 0`with Noise
The clustering algorithm believes there to be a cluster within this group

The DBScan clustering algorithm shows two clusters (indicated by arrows)
![](../../assets/experiments/mesh/noise_anomaly_cluster.png)

Using radius based outlier detection, the red points are all identified as outliers while gray points are all inliers
![](../../assets/experiments/mesh/noise_anomaly_outlier.png)

The point cloud now gets processed to remove all outlier points and the poission mesh recomputed
![](../../assets/experiments/mesh/noise_anomaly_mesh.png)